In [163]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import os 
from glob import glob 

from PIL import Image, ImageOps
from tqdm import tqdm 
import albumentations as A
import cv2 
from augraphy import *
import random
from torchvision import transforms

In [164]:
df = pd.read_csv('../data/train.csv')

In [165]:
H_flip = A.HorizontalFlip(always_apply=True, p=1)
V_flip = A.VerticalFlip(always_apply=True, p=1)

In [166]:
ids = []
targets = []
data_path = '../data/train/'
for index, ID, target in tqdm(df.itertuples(), desc='Image flip', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))

    # V flip
    id_vflip = 'Vflip_'+ ID
    ids.append(id_vflip)
    targets.append(target)
    vflip_image = V_flip(image=image)['image']
    Image.fromarray(vflip_image).save(os.path.join(data_path, id_vflip))
    
    # H flip
    id_hflip = 'Hflip_'+ ID
    ids.append(id_hflip)
    targets.append(target)
    hflip_image = H_flip(image=image)['image']
    Image.fromarray(hflip_image).save(os.path.join(data_path, id_hflip))

    # HV flip
    id_hvflip = 'HVflip_'+ ID
    ids.append(id_hvflip)
    targets.append(target)
    hvflip_image = V_flip(image=H_flip(image=image)['image'])['image']
    Image.fromarray(hvflip_image).save(os.path.join(data_path, id_hvflip))
    
flip_data = {
    'ID' : ids,
    'target' : targets
}
flip_df = pd.DataFrame(flip_data)    
df = pd.concat([df, flip_df])

Image flip: 1570it [00:28, 54.90it/s]


In [167]:
ink_phase = []

In [168]:
paper_phase_1 = [
    Brightness(brightness_range=(0.5, 1.5), min_brightness_value=(120, 150), p=0.5),
]
paper_phase_2 = [
    ShadowCast(
        shadow_side = "random",
        shadow_vertices_range = (2, 3),
        shadow_width_range=(0.5, 0.8),
        shadow_height_range=(0.5, 0.8),
        shadow_color = (0, 0, 0),
        shadow_opacity_range=(0.5,0.6),
        shadow_iterations_range = (1,2),
        shadow_blur_kernel_range = (101, 301),
        p=0.5
    )
]

In [169]:
post_phase_1 = [
    Scribbles(
        scribbles_type="random",
        scribbles_location="random",
        scribbles_size_range=(50, 100),
        scribbles_count_range=(1, 6),
        scribbles_thickness_range=(1, 3),
        scribbles_brightness_change=[32, 64, 128],
        scribbles_text="random",
        scribbles_text_font="random",
        scribbles_text_rotate_range=(0, 360),
        scribbles_lines_stroke_count_range=(1, 6),
    ),
]
post_phase_2 = [
    Folding(
        fold_count=2,
        fold_noise=0.0,
        fold_angle_range = (-360, 360),
        gradient_width=(0.1, 0.2),
        gradient_height=(0.01, 0.1),
        backdrop_color = (255, 255, 255),
    ),
]
post_phase_3 = [
    NoiseTexturize(
        sigma_range=(13, 18),
        turbulence_range=(2, 5),
        texture_width_range=(250, 500),
        texture_height_range=(250, 500),
    ),
]
post_phase_4 = [
    BrightnessTexturize(texturize_range=(0.6, 0.99), deviation=(0.3)),
]
post_phase_5 = [
    SubtleNoise(subtle_range=50),
]


In [170]:
pipeline1 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_1, post_phase=post_phase_1)
pipeline2 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_1, post_phase=post_phase_2)
pipeline3 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_1, post_phase=post_phase_3)
pipeline4 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_1, post_phase=post_phase_4)
pipeline5 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_1, post_phase=post_phase_5)
pipeline6 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_2, post_phase=post_phase_1)
pipeline7 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_2, post_phase=post_phase_2)
pipeline8 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_2, post_phase=post_phase_3)
pipeline9 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_2, post_phase=post_phase_4)
pipeline10 = AugraphyPipeline(ink_phase=ink_phase, paper_phase=paper_phase_2, post_phase=post_phase_5)

pipelines = [pipeline1, pipeline2, pipeline3, pipeline4, pipeline5, pipeline6, pipeline7, pipeline8, pipeline9, pipeline10]

In [171]:
len(pipelines)

10

In [174]:
au_ids = []
au_targets = []
data_path = '../data/train/'
for index, ID, target in tqdm(df.itertuples(), desc='Image transforms', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))

    for i, pipeline in enumerate(pipelines):
        transformed_image = pipeline(image)
        image_ID = f'tf_augraphy_{i}_' + ID 
        au_ids.append(image_ID)
        au_targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))

    if target == 13 :
        for i, pipeline in enumerate(np.random.choice(pipelines, 3)):
            transformed_image = pipeline(image)
            image_ID = f'tf_augraphy_{i + 10}_' + ID 
            au_ids.append(image_ID)
            au_targets.append(target)
            Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))
    elif target == 14 or target == 1:
        for i, pipeline in enumerate(pipelines):
            transformed_image = pipeline(image)
            image_ID = f'tf_augraphy_{i + 10}_' + ID 
            au_ids.append(image_ID)
            au_targets.append(target)
            Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))

aug_data_augraphy = {
    'ID' : au_ids,
    'target' : au_targets
}
augraphy_df = pd.DataFrame(aug_data_augraphy)    
# df = pd.concat([df, augraphy_df])


Image transforms: 6280it [1:19:42,  1.31it/s]


In [180]:
transforms = A.Compose([
    A.OneOf([
        A.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=(255, 255, 255), mask_value=None, always_apply=True, p=1),
        A.ElasticTransform(always_apply=True, p=1, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=1, value=(255, 255, 255), mask_value=None, approximate=False),
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(-0.3, -0.1)),
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(0.1, 0.3)),
    ], p=0.5),
    A.SomeOf([
        A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1),
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=20, p=1),
        A.MultiplicativeNoise(p=1, multiplier=(1, 1.5), per_channel=False),
        A.Equalize(p=1, mode='cv', by_channels=False),
    ], n=2, p=0.4),
    A.CoarseDropout(p=0.4, max_holes=40, max_height=15, max_width=15, min_holes=8, min_height=8, min_width=8),
    A.OneOf([
        A.Blur(blur_limit=(3, 4), p=1),
        A.MotionBlur(blur_limit=(3, 5), p=1),
        A.Downscale(scale_min=0.455, scale_max=0.5, interpolation=2, p=1),
    ], p=0.5),
    A.GaussNoise(var_limit=(400, 1300), per_channel=False, p=0.5),
])

In [179]:
df.target.value_counts()

target
16    400
10    400
0     400
3     400
12    400
8     400
2     400
11    400
7     400
9     400
15    400
5     400
4     400
6     400
13    296
14    200
1     184
Name: count, dtype: int64

In [181]:
ids = []
targets = []
data_path = '../data/train/'
for index, ID, target in tqdm(df.itertuples(), desc='Image augmentation', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))
    if target == 13:
        n = 13
    elif target == 14 or target == 1:
        n = 20
    else:
        n = 10
    for i in range(n):
        transformed_image = transforms(image=image)['image']
        image_ID = f'A_tf{i}_' + ID 
        ids.append(image_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))
    
aug_data = {
    'ID' : ids,
    'target' : targets
}
aug_df = pd.DataFrame(aug_data)    
# df = pd.concat([df, aug_df])

Image augmentation: 6280it [38:13,  2.74it/s]


In [187]:
new_df = pd.concat([df, aug_df, augraphy_df])
new_df

,ID,target
0,002f99746285dfdd.jpg,16
1,008ccd231e1fea5d.jpg,10
2,008f5911bfda7695.jpg,10
3,009235e4c9c07af5.jpg,4
4,00b2f44967580c74.jpg,16
...,...,...
67523,tf_augraphy_5_HVflip_ffc22136f958deb1.jpg,9
67524,tf_augraphy_6_HVflip_ffc22136f958deb1.jpg,9
67525,tf_augraphy_7_HVflip_ffc22136f958deb1.jpg,9
67526,tf_augraphy_8_HVflip_ffc22136f958deb1.jpg,9


In [189]:
new_df.loc[new_df['ID'].str.contains('45f0d2dfc7e47c03.jpg'), 'target'] = 7
new_df.loc[new_df['ID'].str.contains('4620f6e53442f3b6.jpg'), 'target'] = 7
new_df.loc[new_df['ID'].str.contains('aec62dced7af97cd.jpg'), 'target'] = 14
new_df.loc[new_df['ID'].str.contains('c5182ab809478f12.jpg'), 'target'] = 14
new_df.loc[new_df['ID'].str.contains('0583254a73b48ece.jpg'), 'target'] = 10
new_df.loc[new_df['ID'].str.contains('38d1796b6ad99ddd.jpg'), 'target'] = 10
new_df.loc[new_df['ID'].str.contains('1ec14a14bbe633db.jpg'), 'target'] = 7
new_df.loc[new_df['ID'].str.contains('8646f2c3280a4f49.jpg'), 'target'] = 3

In [191]:
new_df.to_csv('../data/new_aug_train.csv', index=False)